# Inject Mocks as Dependencies

## You wrote some code making HTTP requests over the Internet.

In [ ]:
import requests


def do_crazy_stuff():
    return requests.get('http://www.do-something.com/crazy/').text


def do_awesome_stuff():
    return requests.get('http://www.do-something.com/awesome/').text


def do_random_stuff():
    return requests.get('http://www.do-something.com/random/').text

## You notice that you don't have access to the Internet in your tests.
## So you decide to use Mocks.

In [ ]:
from unittest.mock import patch


def test_do_crazy_stuff():
    with patch('requests.get') as response_mock:
        response_mock.text.return_value = 'crazy_stuff'
        assert do_crazy_stuff() == 'crazy_stuff'


def test_do_crazy_stuff():
    with patch('requests.get') as response_mock:
        response_mock.text.return_value = 'awesome_stuff'
        assert do_awesome_stuff() == 'awesome_stuff'


def test_do_random_stuff():
    with patch('requests.get') as response_mock:
        response_mock.text.return_value = 'random_stuff'
        assert do_random_stuff() == 'random_stuff'

## Later on, you discover the amazing Responses library, to mock the Requests library.
## So you refactor your tests.

In [ ]:
import responses


@responses.activate
def test_do_crazy_stuff():
    responses.add(responses.GET, 'http://www.do-something.com/crazy/', body='crazy_stuff')
    assert do_crazy_stuff() == 'crazy_stuff'


@responses.activate
def test_do_awesome_stuff():
    responses.add(responses.GET, 'http://www.do-something.com/awesome/', body='awesome_stuff')
    assert do_awesome_stuff() == 'awesome_stuff'


@responses.activate
def test_do_random_stuff():
    responses.add(responses.GET, 'http://www.do-something.com/random/', body='random_stuff')
    assert do_random_stuff() == 'random_stuff'

## But making synchronous requests is so 90's!
## Now, cool guys use Async everywhere!!!
## So you decide to use Grequests instead, to make async requests using Requests + Gevent.

In [ ]:
import grequests


def do_crazy_stuff():
    return grequests.get('http://www.do-something.com/crazy/').text


def do_awesome_stuff():
    return grequests.get('http://www.do-something.com/awesome/').text


def do_random_stuff():
    return grequests.get('http://www.do-something.com/random/').text

## Incidentally, you have to rewrite again yours Mocks now...

In [ ]:
from unittest.mock import patch


def test_do_crazy_stuff():
    with patch('grequests.get') as response_mock:
        response_mock.text.return_value = 'crazy_stuff'
        assert do_crazy_stuff() == 'crazy_stuff'


def test_do_crazy_stuff():
    with patch('grequests.get') as response_mock:
        response_mock.text.return_value = 'awesome_stuff'
        assert do_awesome_stuff() == 'awesome_stuff'


def test_do_random_stuff():
    with patch('grequests.get') as response_mock:
        response_mock.text.return_value = 'random_stuff'
        assert do_random_stuff() == 'random_stuff'

## Instead, you could have injected your HTTP client as a dependency from the beginning.

In [ ]:
import requests


def do_crazy_stuff(client=requests):
    return client.get('http://www.do-something.com/crazy/').text


def do_awesome_stuff(client=requests):
    return client.get('http://www.do-something.com/awesome/').text


def do_random_stuff(client=requests):
    return client.get('http://www.do-something.com/random/').text

## By writing your own Fake client (a Mock-like).

In [ ]:
from collections import UserDict


class FakeHTTPResponse:
    """Simulate :class:`requests.Response`.

    :param dict data: response's body.
    :param int status_code: response's status code.
    """

    def __init__(self, data=None, status_code=200):
        self.data = data or dict()
        self.status_code = status_code

    @property
    def text(self):
        """Decode response's body."""
        return self.data


class FakeHTTPClient(UserDict):
    """Simulate :mod:`request`'s high-level API."""

    def get(self, url):
        """Retrieve content from :data:`url`.

        :rtype: FakeHTTPResponse
        """
        for endpoint, response in self.data.items():
            if url.endswith(endpoint):
                return FakeHTTPResponse(response)

        return FakeHTTPResponse(status_code=404)

## And injecting your Fake, from your tests, into your code.

In [ ]:
client = FakeHTTPClient()

def test_do_crazy_stuff():
    client['http://www.do-something.com/crazy/'] = 'crazy_stuff'
    assert do_crazy_stuff(client) == 'crazy_stuff'

def test_do_awesome_stuff():
    client['http://www.do-something.com/awesome/'] = 'awesome_stuff'
    assert do_awesome_stuff(client) == 'awesome_stuff'

def test_do_random_stuff():
    client['http://www.do-something.com/random/'] = 'random_stuff'
    assert do_random_stuff(client) == 'random_stuff'

## This way, your test remains independant of your implementation.
## You can, for example, switch back to Grequests.

In [ ]:
import grequests


def do_crazy_stuff(client=grequests):
    # Implementation does not change.


def do_crazy_stuff(client=grequests):
    # Implementation does not change.


def do_crazy_stuff(client=grequests):
    # Implementation does not change.

## Or Requests-Futures.

In [ ]:
import inspect

from requests_futures.sessions import FuturesSession


def do_crazy_stuff(client=FutureSession):
    return _do_something('/crazy/', client)


def do_awesome_stuff(client=FutureSession):
    return _do_something('/awesome/', client)


def do_random_stuff(client=FutureSession):
    return _do_something('/random/', client)


def _do_something(endpoint, client):
    client = client() if inspect.isclass(client) else client
    response = client.get(f'http://www.do-something.com{endpoint}')
    return response.result().text

## And only change your tests configuration (in one place), to reflect the changes.

In [ ]:
class FakeHTTPResponse:
    # Previous implementation stays the same.
    
    def result(self):
        """Return the processed response."""
        return self

## The tests themselves remain unchanged.